# **NYC Trip Fare Analysis - Foundations of Computer Science Project**<br/>

**Master's Degree in Data Science (A.Y. 2024/2025)**<br/>
**University of Milano - Bicocca**<br/>

Students: Sara **Nava** (mat. **870885**), Nicola **Perani** (mat. **864755**)

This notebook presents the project based on the **NYC Trip Fare Analysis** dataset.

**Dataset Description**</br>

The dataset is described as follows (extracted from [Kaggle](https://www.kaggle.com/datasets/diishasiing/revenue-for-cab-drivers)):

- **VendorID**: Unique identifier for the taxi vendor or service provider.
- **tpep_pickup_datetime**: Date and time of passenger pickup.
- **tpep_dropoff_datetime**: Date and time of passenger drop-off.
- **passenger_count**: Number of passengers in the taxi.
- **trip_distance**: Total distance of the trip in miles or kilometers.
- **RatecodeID**: Rate code assigned to the trip, representing fare types.
- **store_and_fwd_flag**: Indicates whether the trip data was stored locally before being forwarded (Y/N).
- **PULocationID**: Unique identifier for the pickup location (zone or area).
- **DOLocationID**: Unique identifier for the drop-off location (zone or area).
- **payment_type**: Passenger's method of payment (e.g., cash, card).
- **fare_amount**: Base fare for the trip.
- **extra**: Additional charges applied during the trip (e.g., night surcharge).
- **mta_tax**: Tax imposed by the Metropolitan Transportation Authority.
- **tip_amount**: Tip given to the driver, if applicable.
- **tolls_amount**: Total tolls charged during the trip.
- **improvement_surcharge**: Surcharge imposed for service improvements.
- **total_amount**: Total fare amount, including all charges and surcharges.
- **congestion_surcharge**: Additional charge for trips taken during high-traffic congestion periods.

**Data Preparation**</br>

Before proceeding with the tasks described in the **README**, it is essential to:
- Set up the working environment.
- Import the required libraries.
- Clean and analyze the data.

## **1. Set Environment and Import Libraries**

Set up the working environment by ensuring that all scripts and notebooks can access the project’s main directories dynamically. The `conf.py` module, located in the conf folder, defines key paths (`MAIN_DIR`, `NOTEBOOK_DIR`, `DATA_DIR`) and adds them to `sys.path`. This allows seamless imports and ensures portability across different environments without requiring hardcoded paths.

In [1]:
import sys
import os

# Add 'conf' folder to sys.path (if not already present)
conf_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'conf'))
if conf_path not in sys.path:
    sys.path.append(conf_path)

# Now import conf module from conf folder
import conf

Now, import main packages necessary for the developement of this project:

In [3]:
# Import libraries
import pandas as pd
import datetime
import time
import numpy as np

## **2. Data Importation and Pre-Processing**

### **2.a Import Data**

Import the dataset present within the `DATA_DIR` folder:

In [4]:
# Read data from the .csv file within data folder
dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')

/var/folders/w7/0xpq0vds7cz2333831nfq5dc0000gn/T/ipykernel_40100/2694177516.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f'{conf.DATA_DIR}/data.csv')


First of all, create a copy of the original dataset for recovery.

In [5]:
# Create a copy of the original dataset
dataset_old = dataset.copy()

Print the **first 5 rows** of the dataset to visualize a sample of data:

In [6]:
dataset.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


### **2.b Format Analysis**

After importing data, a `DtypeWarning` appeared. Print out **columns types** to check the warning (columns with different data types):

In [7]:
print(dataset.dtypes)

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object


Consider the column `store_and_fwd_flag`: 

In [8]:
# Get the unique values present in the 'store_and_fwd_flag' column, to understand the distinct forwarding flags in the dataset
dataset.store_and_fwd_flag.unique()

array(['N', 'Y', nan], dtype=object)

The problem is on 'nan' that are encoded in a wrong way, so handle the missing values correctly and then convert variable dtype into `category`.

In [9]:
# Clean up missing values before conversion into type "category"
dataset['store_and_fwd_flag'] = dataset['store_and_fwd_flag'].replace(["", " ", "NaN", "nan"], pd.NA).astype("category")

# Get again unique categories
dataset.store_and_fwd_flag.unique()

['N', 'Y', NaN]
Categories (2, object): ['N', 'Y']

Moreover, dates must be converted into the proper `datetime` format:

In [10]:
# Convert datetime fields to proper format
dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'], errors='coerce') # errors = 'coerce' to avoid anomaly
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'], errors='coerce')

Check new dtypes:

In [11]:
# Print formats
print(dataset[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag']].dtypes)

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
store_and_fwd_flag             category
dtype: object


Now, considering how data are described within the Kaggle datacard, try to understand if adjustments are required. 

It is important that numerical IDs, counters and numeric categorical variables are converted into **integer numbers**. With the format `Int64` it is possible to handle null values for integer data without converting them into `float64`. 

For numerical IDs and categorical columns, is then necessary to convert them into **categorical**.

Variables which require a conversion are `VendorID`, `RatecodeID`, `DOLocationID`, `PULocationID` (*numerical IDs*), `payment_type` (*categorical column*) and `passenger_count` (*counter column*).

In [12]:
# List of columns to modify
col_to_modify = ['VendorID', 'passenger_count', 'RatecodeID', 'payment_type', 'DOLocationID', 'PULocationID']

# Function to convert data types for optimization
def type_conversion(col_name):
    # Convert the column to nullable integers (Int64) to handle NaN values
    column_new = dataset[col_name].astype('Int64')
    
    # Convert to 'category' data type if it's not 'passenger_count'
    # 'category' type reduces memory usage for columns with repeated values
    if col_name != 'passenger_count':
        column_new = column_new.astype('category')

    return column_new

# Apply type conversion to each column in the list
for col in col_to_modify:
    dataset[col] = type_conversion(col)

Print how variable dtypes are changed:

In [13]:
for col in list(dataset_old.columns):
    print('Column :', col, '| before:', dataset_old[col].dtype, ' -> after:', dataset[col].dtype)

Column : VendorID | before: float64  -> after: category
Column : tpep_pickup_datetime | before: object  -> after: datetime64[ns]
Column : tpep_dropoff_datetime | before: object  -> after: datetime64[ns]
Column : passenger_count | before: float64  -> after: Int64
Column : trip_distance | before: float64  -> after: float64
Column : RatecodeID | before: float64  -> after: category
Column : store_and_fwd_flag | before: object  -> after: category
Column : PULocationID | before: int64  -> after: category
Column : DOLocationID | before: int64  -> after: category
Column : payment_type | before: float64  -> after: category
Column : fare_amount | before: float64  -> after: float64
Column : extra | before: float64  -> after: float64
Column : mta_tax | before: float64  -> after: float64
Column : tip_amount | before: float64  -> after: float64
Column : tolls_amount | before: float64  -> after: float64
Column : improvement_surcharge | before: float64  -> after: float64
Column : total_amount | before

Finally, create a rowID:

In [14]:
# Add rowID using .loc to avoid SettingWithCopyWarning
dataset['rowID'] = range(1, len(dataset) + 1)

# Reorder columns to place 'rowID' as the first column
dataset = dataset[['rowID'] + [col for col in dataset.columns if col != 'rowID']]

### **2.c Consistency Analysis**

The function `check_anomalies` is defined to identify potential data anomalies in specific columns of a dataset. Anomalies are defined here as values that are less than 0, as negative values might not be valid for certain fields (e.g., amounts or distances).

The function operates by first removing any `NaN` (missing) values from the specified column using `dropna()`. It then filters for any values in the given column that are less than 0, as these are considered anomalies. If no anomalies are found, a message is printed indicating that the column is clean. If anomalies are detected, the function prints the number of anomalies found and displays the first few rows of the anomalies for inspection.

A list of columns to check for anomalies is defined, which includes common fields such as `fare_amount`, `trip_distance`, and `passenger_count`. The function is applied to each of these columns by iterating through the list and calling `check_anomalies` for each column.

In [15]:
# Function to check for anomalies in a given column
def check_anomalies(dataset, column_name):
    # Exclude NaN values before checking for anomalies
    filtered_dataset = dataset.dropna(subset=[column_name])

    # Identify anomalies where values are less than 0
    anomalies = filtered_dataset[filtered_dataset[column_name] < 0]

    # Print results
    if anomalies.empty:
        print(f"No anomalies found in '{column_name}'.")
    else:
        print(f"Found {len(anomalies)} anomalies in '{column_name}'.")
        display(anomalies.head())  # Display the first few rows of anomalies

# List of columns to check for anomalies
columns_to_check = [
    'passenger_count', 
    'trip_distance', 
    'fare_amount', 
    'mta_tax', 
    'tip_amount', 
    'tolls_amount', 
    'improvement_surcharge', 
    'total_amount', 
    'congestion_surcharge'
]

# Iterate over the columns and check for anomalies
for column in columns_to_check:
    check_anomalies(dataset, column)

No anomalies found in 'passenger_count'.
Found 2338 anomalies in 'trip_distance'.


,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
273001,273002,2,2020-01-02 17:43:57,2020-01-02 17:49:48,1,-0.88,5,N,7,193,1,8.20,0.0,0.5,1.8,0.00,0.3,12.75,0.0
277810,277811,2,2020-01-02 17:50:27,2020-01-02 17:54:55,1,-0.88,5,N,193,193,1,7.20,0.0,0.5,1.6,0.00,0.3,11.55,0.0
282887,282888,2,2020-01-02 18:12:24,2020-01-02 18:15:48,1,-0.71,5,N,193,193,1,7.20,0.0,0.5,1.0,0.00,0.3,10.95,0.0
787926,787927,2,2020-01-05 15:36:50,2020-01-05 16:07:07,1,-9.29,5,N,225,233,2,21.41,0.0,0.5,0.0,12.24,0.3,34.45,0.0
905236,905237,2,2020-01-06 10:43:08,2020-01-06 10:49:59,1,-1.79,5,N,145,193,1,9.20,0.0,0.5,2.0,0.00,0.3,13.95,0.0


Found 19505 anomalies in 'fare_amount'.


,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
160,161,2,2020-01-01 00:54:28,2020-01-01 00:58:41,1,0.60,1,N,170,137,3,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-8.3,-2.5
372,373,2,2020-01-01 00:57:13,2020-01-01 00:58:05,1,0.11,1,N,260,260,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,0.0
667,668,2,2020-01-01 00:36:31,2020-01-01 00:37:23,3,0.18,1,N,141,141,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-6.8,-2.5
677,678,2,2020-01-01 00:23:15,2020-01-01 00:23:54,2,0.09,1,N,114,114,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-6.3,-2.5
1007,1008,2,2020-01-01 00:13:39,2020-01-01 00:24:26,2,1.26,1,N,249,68,4,-8.5,-0.5,-0.5,0.0,0.0,-0.3,-12.3,-2.5


Found 19028 anomalies in 'mta_tax'.


...

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
160,161,2,2020-01-01 00:54:28,2020-01-01 00:58:41,1,0.60,1,N,170,137,3,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-8.3,-2.5
372,373,2,2020-01-01 00:57:13,2020-01-01 00:58:05,1,0.11,1,N,260,260,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,0.0
667,668,2,2020-01-01 00:36:31,2020-01-01 00:37:23,3,0.18,1,N,141,141,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-6.8,-2.5
677,678,2,2020-01-01 00:23:15,2020-01-01 00:23:54,2,0.09,1,N,114,114,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-6.3,-2.5
1007,1008,2,2020-01-01 00:13:39,2020-01-01 00:24:26,2,1.26,1,N,249,68,4,-8.5,-0.5,-0.5,0.0,0.0,-0.3,-12.3,-2.5


Found 19505 anomalies in 'total_amount'.


,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
160,161,2,2020-01-01 00:54:28,2020-01-01 00:58:41,1,0.60,1,N,170,137,3,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-8.3,-2.5
372,373,2,2020-01-01 00:57:13,2020-01-01 00:58:05,1,0.11,1,N,260,260,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,0.0
667,668,2,2020-01-01 00:36:31,2020-01-01 00:37:23,3,0.18,1,N,141,141,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-6.8,-2.5
677,678,2,2020-01-01 00:23:15,2020-01-01 00:23:54,2,0.09,1,N,114,114,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-6.3,-2.5
1007,1008,2,2020-01-01 00:13:39,2020-01-01 00:24:26,2,1.26,1,N,249,68,4,-8.5,-0.5,-0.5,0.0,0.0,-0.3,-12.3,-2.5


Found 15426 anomalies in 'congestion_surcharge'.


,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
160,161,2,2020-01-01 00:54:28,2020-01-01 00:58:41,1,0.60,1,N,170,137,3,-4.5,-0.5,-0.5,0.0,0.0,-0.3,-8.3,-2.5
667,668,2,2020-01-01 00:36:31,2020-01-01 00:37:23,3,0.18,1,N,141,141,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-6.8,-2.5
677,678,2,2020-01-01 00:23:15,2020-01-01 00:23:54,2,0.09,1,N,114,114,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-6.3,-2.5
1007,1008,2,2020-01-01 00:13:39,2020-01-01 00:24:26,2,1.26,1,N,249,68,4,-8.5,-0.5,-0.5,0.0,0.0,-0.3,-12.3,-2.5
1449,1450,2,2020-01-01 00:18:16,2020-01-01 00:18:42,1,0.03,1,N,164,164,4,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-6.3,-2.5


Since the reasons behind these anomalies are unknown, it would be **inappropriate** to determine a fixed approach for handling them or to make incorrect assumptions. The best approach is to leave the dataset unchanged and, if specific requests involve variables affected by anomalies, **address them on a case-by-case basis** using the most suitable method.

## **3. Assignements**

### 3.1. Extract all trips with `trip_distance` larger than 50

Filter the dataset considering only those rows where the `trip_distance` is greather than 50.

**Note**: this column was affected by anomalies. Since the filter keep only distances longer than 50, anomalies don't impact results.

In [16]:
# Extract all trip with distance > 50
dataset[dataset['trip_distance'] > 50]

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
23842,23843,2,2020-01-01 01:53:07,2020-01-01 03:54:41,1,52.30,5,N,262,265,1,300.00,0.00,0.0,61.78,6.12,0.3,370.70,2.5
39013,39014,2,2020-01-01 02:05:07,2020-01-01 03:03:10,1,51.23,5,N,264,264,1,329.00,0.00,0.5,100.78,6.12,0.3,436.70,0.0
41620,41621,1,2020-01-01 03:05:54,2020-01-01 04:16:26,1,53.80,5,N,132,265,1,250.00,0.00,0.0,53.35,16.62,0.3,320.27,0.0
58262,58263,2,2020-01-01 05:36:12,2020-01-01 06:40:06,1,55.23,5,N,132,265,2,170.00,0.00,0.5,0.00,18.26,0.3,189.06,0.0
63024,63025,2,2020-01-01 07:40:30,2020-01-01 08:40:01,1,54.19,5,N,132,265,1,230.00,0.00,0.0,0.00,12.24,0.3,242.54,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6326169,6326170,2,2020-01-31 22:47:26,2020-01-31 23:49:14,1,51.83,5,N,132,265,1,220.00,0.00,0.5,48.96,23.99,0.3,293.75,0.0
6331181,6331182,2,2020-01-31 23:45:36,2020-02-01 01:00:25,5,57.99,4,N,107,265,1,245.00,0.50,0.5,38.24,6.12,0.3,293.16,2.5
6333801,6333802,2,2020-01-31 23:24:16,2020-02-01 01:32:56,1,52.97,4,N,264,265,1,227.00,0.50,0.5,46.16,0.00,0.3,276.96,2.5
6397132,6397133,NaN,2020-01-28 11:54:00,2020-01-28 19:35:00,<NA>,60.36,NaN,NaN,17,61,NaN,12.04,0.00,0.5,0.00,12.24,0.3,25.08,0.0


### 3.2 Extract all trips where `payment_type` is missing

Filter only those rows where `payment_type` is missing, using the `isna()` module:

In [17]:
# Trips with payment_type missings
dataset[dataset['payment_type'].isna()]

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,6339568,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,<NA>,13.69,NaN,NaN,136,232,NaN,51.05,2.75,0.5,0.0,0.00,0.3,54.60,0.0
6339568,6339569,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,<NA>,3.42,NaN,NaN,121,9,NaN,27.06,2.75,0.0,0.0,0.00,0.3,30.11,0.0
6339569,6339570,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,<NA>,2.20,NaN,NaN,197,216,NaN,24.36,2.75,0.5,0.0,0.00,0.3,27.91,0.0
6339570,6339571,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,<NA>,0.84,NaN,NaN,262,236,NaN,26.08,2.75,0.5,0.0,0.00,0.3,29.63,0.0
6339571,6339572,NaN,2020-01-01 08:21:00,2020-01-01 08:38:00,<NA>,7.24,NaN,NaN,45,142,NaN,25.28,2.75,0.5,0.0,0.00,0.3,28.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,6405004,NaN,2020-01-31 22:51:00,2020-01-31 23:22:00,<NA>,3.24,NaN,NaN,237,234,NaN,17.59,2.75,0.5,0.0,0.00,0.3,21.14,0.0
6405004,6405005,NaN,2020-01-31 22:10:00,2020-01-31 23:26:00,<NA>,22.13,NaN,NaN,259,45,NaN,46.67,2.75,0.5,0.0,12.24,0.3,62.46,0.0
6405005,6405006,NaN,2020-01-31 22:50:07,2020-01-31 23:17:57,<NA>,10.51,NaN,NaN,137,169,NaN,48.85,2.75,0.0,0.0,0.00,0.3,51.90,0.0
6405006,6405007,NaN,2020-01-31 22:25:53,2020-01-31 22:48:32,<NA>,5.49,NaN,NaN,50,42,NaN,27.17,2.75,0.0,0.0,0.00,0.3,30.22,0.0


### 3.3 For each (`PULocationID`, `DOLocationID`) pair, determine the number of trips

To calculate the number of trips for each unique pair of pickup location (`PULocationID`) and dropoff location (`DOLocationID`), use the groupby operation, and `size()` counts the number of trips in each group. The result is then reset into a DataFrame with a new column trip_count representing the count of trips for each location pair.

**Note**: for the entire project, everytime it is necessary to apply a `groupby`, specify the statement `observed=False` due to future warnings (except when it is explicitly necessary to set it as `True`) .

In [18]:
# Group the dataset by 'PULocationID' and 'DOLocationID', counting the number of trips for each pair

dataset.groupby(['PULocationID', 'DOLocationID'], 
                observed=False # ensures that only pairs that exist in the data are included (no missing combinations)
                ).size().reset_index(name='trip_count') # converts the result back to the DataFrame and names the new column 'trip_count'

,PULocationID,DOLocationID,trip_count
0,1,1,638
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0
...,...,...,...
68377,265,261,1
68378,265,262,0
68379,265,263,4
68380,265,264,317


### 3.4 Save all rows with missing `VendorID`, `passenger_count`, `store_and_fwd_flag`, `payment_type` in a new dataframe called `bad`, and remove those rows from the original dataframe

There are two possibilities:
-	`.any(axis=1)`: Removes rows where **at least one** of the specified columns is **NaN**.
-	`.all(axis=1)`: Removes rows where **all specified columns** are **NaN** at the same time.

In this case, the proper solution is `.any(axis=1)`:

In [19]:
# Filter rows where any of the specified columns ('VendorID', 'passenger_count', 'payment_type', 'store_and_fwd_flag') contain NaN values
# 'isna()' checks for missing values, and 'any(axis=1)' ensures that any row with NaN in any of these columns is included
bad = dataset[dataset[['VendorID', 'passenger_count', 'payment_type', 'store_and_fwd_flag']].isna().any(axis=1)]
bad

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6339567,6339568,NaN,2020-01-01 08:51:00,2020-01-01 09:19:00,<NA>,13.69,NaN,NaN,136,232,NaN,51.05,2.75,0.5,0.0,0.00,0.3,54.60,0.0
6339568,6339569,NaN,2020-01-01 08:38:43,2020-01-01 08:51:08,<NA>,3.42,NaN,NaN,121,9,NaN,27.06,2.75,0.0,0.0,0.00,0.3,30.11,0.0
6339569,6339570,NaN,2020-01-01 08:27:00,2020-01-01 08:32:00,<NA>,2.20,NaN,NaN,197,216,NaN,24.36,2.75,0.5,0.0,0.00,0.3,27.91,0.0
6339570,6339571,NaN,2020-01-01 08:46:00,2020-01-01 08:57:00,<NA>,0.84,NaN,NaN,262,236,NaN,26.08,2.75,0.5,0.0,0.00,0.3,29.63,0.0
6339571,6339572,NaN,2020-01-01 08:21:00,2020-01-01 08:38:00,<NA>,7.24,NaN,NaN,45,142,NaN,25.28,2.75,0.5,0.0,0.00,0.3,28.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405003,6405004,NaN,2020-01-31 22:51:00,2020-01-31 23:22:00,<NA>,3.24,NaN,NaN,237,234,NaN,17.59,2.75,0.5,0.0,0.00,0.3,21.14,0.0
6405004,6405005,NaN,2020-01-31 22:10:00,2020-01-31 23:26:00,<NA>,22.13,NaN,NaN,259,45,NaN,46.67,2.75,0.5,0.0,12.24,0.3,62.46,0.0
6405005,6405006,NaN,2020-01-31 22:50:07,2020-01-31 23:17:57,<NA>,10.51,NaN,NaN,137,169,NaN,48.85,2.75,0.0,0.0,0.00,0.3,51.90,0.0
6405006,6405007,NaN,2020-01-31 22:25:53,2020-01-31 22:48:32,<NA>,5.49,NaN,NaN,50,42,NaN,27.17,2.75,0.0,0.0,0.00,0.3,30.22,0.0


Now, clean `dataset` from rows saved in `bad` dataframe. 

In [20]:
# Filter rows by checking if 'rowID' is in `bad`
dataset = dataset[~dataset.index.isin(bad.index)]

# Show the cleaned dataset
dataset

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.20,1,N,238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,2,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.20,1,N,239,238,1,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,3,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.60,1,N,238,238,1,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,4,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.80,1,N,238,151,1,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,5,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,6339563,2,2020-01-31 23:38:07,2020-01-31 23:52:21,1,2.10,1,N,163,246,1,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,6339564,2,2020-01-31 23:00:18,2020-01-31 23:19:18,1,2.13,1,N,164,79,1,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,6339565,2,2020-01-31 23:24:22,2020-01-31 23:40:39,1,2.55,1,N,79,68,1,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,6339566,2,2020-01-31 23:44:22,2020-01-31 23:54:00,1,1.61,1,N,100,142,2,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


### 3.5 Add a duration column storing how long each trip has taken (use `tpep_pickup_datetime`, `tpep_dropoff_datetime`)

Since `tpep_pickup_datetime` is the date and time when the passenger was picked up and `tpep_dropoff_datetime` is the date and time when the passenger was dropped off, the duration can be simply computed as this difference: `tpep_dropoff_datetime` -  `tpep_pickup_datetime`.

In [21]:
# Save within the column trip_duration the difference between dropoff time and pickup time
dataset['trip_duration'] = dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']

# Display results
dataset[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_duration']]

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_duration
0,2020-01-01 00:28:15,2020-01-01 00:33:03,0 days 00:04:48
1,2020-01-01 00:35:39,2020-01-01 00:43:04,0 days 00:07:25
2,2020-01-01 00:47:41,2020-01-01 00:53:52,0 days 00:06:11
3,2020-01-01 00:55:23,2020-01-01 01:00:14,0 days 00:04:51
4,2020-01-01 00:01:58,2020-01-01 00:04:16,0 days 00:02:18
...,...,...,...
6339562,2020-01-31 23:38:07,2020-01-31 23:52:21,0 days 00:14:14
6339563,2020-01-31 23:00:18,2020-01-31 23:19:18,0 days 00:19:00
6339564,2020-01-31 23:24:22,2020-01-31 23:40:39,0 days 00:16:17
6339565,2020-01-31 23:44:22,2020-01-31 23:54:00,0 days 00:09:38


The result is a `Timedelta` column, which represents the time difference as **number of days, hours, minutes and seconds**.

### 3.6 For each pickup location, determine how many trips have started there 

The pickup location is stored within `PULocationID` column. It is simply necessary to compute values distribution with the respect to this column, grouping by `PULocationID` and then applying `size()` function to count trips for each pickup location.

The number of trips is saved within the column `trip_count`:

In [22]:
# Group the dataset by 'PULocationID', counting the number of trips for each pickup location
# 'observed=False' ensures that only locations that exist in the data are included
# 'size()' counts the number of rows (trips) in each group
# 'reset_index()' converts the result back to a DataFrame and names the new column 'trip_count'

dataset.groupby('PULocationID', observed=False).size().reset_index(name='trip_count')

,PULocationID,trip_count
0,1,753
1,2,3
2,3,70
3,4,9902
4,5,39
...,...,...
256,261,34229
257,262,85591
258,263,123997
259,264,43779


### 3.7 Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)

The following procedure clusters the pickup times of the day into 30-minute intervals. The `pd.date_range()` function generates time bins from 00:00:00 to 23:59:59 with 30-minute intervals. Then, labels are created to represent each time interval in a readable format (e.g., “02:00 - 02:30”). Finally, the `pd.cut()` function assigns each pickup time to its corresponding 30-minute interval and creates a new column `pickup_time_interval` in the dataset.

Note: the `pd.date_range` excludes the interval **23:30 - 00:00**, thus append it manually.

In [23]:
# Generate time intervals (30 min bins)
bins = pd.date_range(start='00:00:00', end='23:59:59', freq='30min').time
# Include also the interval "23:30 - 23:59"
bins = np.append(bins, datetime.time(23, 59, 59))

# Define labels (e.g., "02:00-02:30", ...)
labels = [f"{bins[i].strftime('%H:%M')} - {bins[i+1].strftime('%H:%M')}" for i in range(len(bins)-1)]
labels[-1] = '23:30 - 00:00'

In [24]:
# Assign each pickup time to its corresponding 30-minute interval with pandas .cut module
dataset['pickup_time_interval'] = pd.cut(dataset['tpep_pickup_datetime'].dt.time, bins=bins, labels=labels, include_lowest=True)

Ensure that each row is correctly classified within a pickup time interval:

In [25]:
# Check the presence of null values within pickup_time_interval
dataset[dataset['pickup_time_interval'].isna()]

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval


Print results:

In [26]:
# Print results
dataset.head()

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval
0,1,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.2,1,N,238,239,...,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,0 days 00:04:48,00:00 - 00:30
1,2,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.2,1,N,239,238,...,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,0 days 00:07:25,00:30 - 01:00
2,3,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.6,1,N,238,238,...,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,0 days 00:06:11,00:30 - 01:00
3,4,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.8,1,N,238,151,...,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,0 days 00:04:51,00:30 - 01:00
4,5,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.0,1,N,193,193,...,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,0 days 00:02:18,00:00 - 00:30


### 3.8 For each interval, determine the average number of passengers and the average fare amount

Proceed with grouping the dataset by the pickup time intervals (`pickup_time_interval`) and calculate the **average passenger count** and **average fare amount** for each interval. The `agg()` function compute the mean of the `passenger_count` and `fare_amount` columns within each time interval.

**Note**: `fare_amount` presents anomalies. It could be appropriate to **remove negative values** before computing average (on fare amount, not on passenger count). 

In [27]:
# Group the dataset by 'pickup_time_interval'
dataset.groupby('pickup_time_interval', observed=False).agg(
    avg_passenger_count=('passenger_count', 'mean'),  # Compute mean on full dataset
    avg_fare_amount=('fare_amount', lambda x: x[x >= 0].mean())  # Exclude negatives only here
).reset_index()

,pickup_time_interval,avg_passenger_count,avg_fare_amount
0,00:00 - 00:30,1.572854,13.636719
1,00:30 - 01:00,1.584273,13.336949
2,01:00 - 01:30,1.578807,12.827622
3,01:30 - 02:00,1.589424,12.395508
4,02:00 - 02:30,1.587403,12.223327
5,02:30 - 03:00,1.587833,12.204906
6,03:00 - 03:30,1.581925,12.705616
7,03:30 - 04:00,1.585897,13.296230
8,04:00 - 04:30,1.580107,14.434104
9,04:30 - 05:00,1.516206,16.658947


### 3.9 For each payment type and each interval, determine the average fare amount

Proceed with grouping the dataset by the payment type (`payment_type`) and by the pickup time intervals (`pickup_time_interval`) and calculate the average fare amount for each group. The `agg()` function compute the **mean** of the `fare_amount` column within each pair `payment_type` and `pickup_time_interval`.

**Note**: also here, it is necessary to remove negative values of `fare_amount`.

In [28]:
# Group the dataset by 'payment_type' and 'pickup_time_interval'
avg_fare = dataset[dataset.fare_amount >= 0].groupby(
    ['payment_type', 'pickup_time_interval'], 
    observed=False).agg(avg_fare_amount=('fare_amount', 'mean') # Compute the average fare amount for each interval
                        ).reset_index()                         # 'reset_index()' converts the result back to a DataFrame

avg_fare

,payment_type,pickup_time_interval,avg_fare_amount
0,1,00:00 - 00:30,13.868641
1,1,00:30 - 01:00,13.473027
2,1,01:00 - 01:30,12.822628
3,1,01:30 - 02:00,12.358248
4,1,02:00 - 02:30,12.008941
...,...,...,...
235,5,21:30 - 22:00,NaN
236,5,22:00 - 22:30,NaN
237,5,22:30 - 23:00,NaN
238,5,23:00 - 23:30,NaN


Where `avg_fare_amount` is `NaN`, it means that no fare amount is registered for `payment_type`-`pickup_time_interval` pair.

### 3.10 For each payment type, determine the interval when the average fare amount is maximum

Data are grouped by the payment type (`payment_type`) and `idxmax()` find the **index of the row** in which the **average fare amount** (`avg_fare_amount`) assumes its **highest value within each group**. Then, `loc[]` selects the corresponding rows, returning the *interval with the highest average fare* *for each payment method*.

**Note**: since the reference dataset is `avg_fare`, anomalies on `fare_amount` have been already excluded.

In [29]:
# Find the interval with the maximum average fare for each payment type
avg_fare.loc[avg_fare.groupby('payment_type', observed=False)['avg_fare_amount'].idxmax()]

,payment_type,pickup_time_interval,avg_fare_amount
10,1,05:00 - 05:30,21.260986
58,2,05:00 - 05:30,14.987023
105,3,04:30 - 05:00,16.555657
154,4,05:00 - 05:30,24.230667
227,5,17:30 - 18:00,0.000000


It can be observed that, except for `payment_type=5`, the **average fare amount** is **higher** during **early morning pickup intervals**, particularly around 4:30 and 5:30.

### 3.11 For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum

Initially, the dataset is grouped by the payment_type (`payment_type`) and by the pickup time interval (`pickup_time_interval`), calculating the **total amount of tips and fares** within each group. 

Then a new column, `tot_amount_ratio`, is created by **dividing the amounts obtained before** (respectively, `tot_tip_amount` and `tot_fare_amount`) by eachother, handling potential division-by-zero errors by **replacing null values** with `NaN`. 

After removing rows with `NaN` values in the ratio column, the **index of the row** where the **overall ratio** (`tot_amount_ratio`) **reaches its maximum value** is identified for each payment method using `idxmax()`. Finally, `loc[]` is used to select the corresponding rows, returning the **interval with the highest tip-to-fare ratio for each payment method**.

**Note**: The sum of `tip_amount` and `fare_amount` is handled separately, removing negative values only for each respective column, due to anomalies.

In [30]:
# Group by payment_type and pickup_time_interval, and for each pair count the total tip and fare amounts
ratio_amount_df = dataset.groupby(['payment_type', 'pickup_time_interval'], observed=False).agg(
    tot_tip_amount=('tip_amount', lambda x: x[x >= 0].sum()),  # Exclude negative tip amounts
    tot_fare_amount=('fare_amount', lambda x: x[x >= 0].sum())  # Exclude negative fare amounts
).reset_index()

# Now, get the ratio between total tip and fare amount for each pair
ratio_amount_df['tot_amount_ratio'] = ratio_amount_df['tot_tip_amount'] / ratio_amount_df['tot_fare_amount'].replace(0, np.nan)  # Avoid division by zero

# Drop rows where tip_fare_ratio is NaN before using idxmax(), to avoid errors
ratio_amount_df = ratio_amount_df.dropna(subset=['tot_amount_ratio'])

# Finally, get the interval for each payment type where the overall ratio is maximum
ratio_amount_df.loc[ratio_amount_df.groupby('payment_type', observed=True)['tot_amount_ratio'].idxmax()]

,payment_type,pickup_time_interval,tot_tip_amount,tot_fare_amount,tot_amount_ratio
37,1,18:30 - 19:00,485536.47,1998420.20,0.242960
58,2,05:00 - 05:30,15.00,110094.67,0.000136
138,3,21:00 - 21:30,41.65,7621.32,0.005465
150,4,03:00 - 03:30,47.34,1178.10,0.040183


Why do rows with `payment_type == 5` not appear? Let’s analyze the dataset rows where this condition is true:

In [31]:
dataset[['payment_type','pickup_time_interval','tip_amount','fare_amount']][dataset.payment_type == 5]

,payment_type,pickup_time_interval,tip_amount,fare_amount
4061635,5,17:30 - 18:00,0.0,0.0


This happens because the only row where the condition is valid has both amounts as null. The ratio between two null values results in `NaN`, since the denominator is zero.

### 3.12 Find the location with the highest average fare amount

There are two types of locations in the dataset: `PULocationID` (pickup location) and `DOLocationID` (drop-off location). Since it’s unclear whether the fare amount is primarily determined by the pickup or the drop-off location, calculate the **average fare amount for each location**, considering *both its role* as a *pickup location* and as a *drop-off location*.

In [32]:
locations = pd.concat([
    # Group by pickup location (PULocationID) and compute the average fare amount
    dataset.groupby('PULocationID', observed=False).agg(avg_fare=('fare_amount', 'mean'))
    .reset_index().rename(columns={'PULocationID': 'LocationID'}),  # Rename column for consistency
    
    # Group by drop-off location (DOLocationID) and compute the average fare amount
    dataset.groupby('DOLocationID', observed=False).agg(avg_fare=('fare_amount', 'mean'))
    .reset_index().rename(columns={'DOLocationID': 'LocationID'})  # Rename column for consistency
])

# Group by location ID and compute the overall average fare across both pickup and drop-off locations
avg_fare_loc = locations.groupby(['LocationID'], observed=False)['avg_fare'].mean().reset_index()

# Find the location with the highest average fare
highest_fare = avg_fare_loc.loc[avg_fare_loc['avg_fare'].idxmax()]

print(f"Location with Highest Avg Fare: {highest_fare['LocationID']}, Avg Fare: {highest_fare['avg_fare']:.2f}")

Location with Highest Avg Fare: 204.0, Avg Fare: 93.18


The location with the **highest average fare** is the **204**, with an average fare of **93.18**.

### 3.13 Build a new dataframe (called `common`) where, for each pickup location we keep all trips to the 5 most common destinations (i.e. each pickup location can have different common destinations)

Proceed by **grouping the dataset** by pickup location ID and drop off location ID (`PULocationID`, `DOLocationID`) to **count the number of trips for each pickup-drop-off pair**. 

Once the counts are computed, the **data is sorted** first by pickup location ID (`PULocationID`) and then by trip frequency (`race_count`) in descending order to rank the most common destinations for each origin. 

A **counter** is introduced within each `PULocationID` group to **assign a rank to the destinations**, ensuring that only the **top five most common locations are kept**. Then the filtered dataset is merged back with the original one, keeping only the trips to the most common locations.

In [33]:
# Group by PULocationID and DOLocationID to calculate the count of trips for each pair
common_destinations = dataset.groupby(['PULocationID', 'DOLocationID'], observed=True # Keep only observed pairs
                                      ).size().reset_index(name='race_count')

# Sort by PULocationID and race_count to get the most common destinations for each pickup location
common_destinations = common_destinations.sort_values(by=['PULocationID', 'race_count'], ascending=[True, False])

# Add a counter within each PULocationID group
common_destinations['counter'] = common_destinations.groupby('PULocationID', observed=False).cumcount() + 1

# Filter the common_destinations to keep only the top 5 destinations per PULocationID
common_destinations = common_destinations[common_destinations['counter'] <= 5].drop(columns=['counter'])

Create the dataset `common` by keeping only the five common destinations:

In [34]:
# Merge this filtered data back to the original dataset to get the corresponding rows
common = dataset.merge(common_destinations[['PULocationID', 'DOLocationID']], 
                       on=['PULocationID', 'DOLocationID'],
                       how = 'inner')

In [35]:
common

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_duration,pickup_time_interval
0,1,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1,1.20,1,N,238,239,...,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,0 days 00:04:48,00:00 - 00:30
1,2,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1,1.20,1,N,239,238,...,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,0 days 00:07:25,00:30 - 01:00
2,3,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1,0.60,1,N,238,238,...,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,0 days 00:06:11,00:30 - 01:00
3,4,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1,0.80,1,N,238,151,...,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,0 days 00:04:51,00:30 - 01:00
4,5,2,2020-01-01 00:01:58,2020-01-01 00:04:16,1,0.00,1,N,193,193,...,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,0 days 00:02:18,00:00 - 00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956034,6339554,2,2020-02-01 00:00:37,2020-02-01 00:11:59,1,1.18,1,N,186,170,...,8.5,0.5,0.5,2.46,0.0,0.3,14.76,2.5,0 days 00:11:22,00:00 - 00:30
1956035,6339555,2,2020-01-31 22:59:01,2020-01-31 23:06:15,3,0.90,1,N,238,239,...,6.5,0.5,0.5,0.00,0.0,0.3,10.30,2.5,0 days 00:07:14,22:30 - 23:00
1956036,6339557,2,2020-01-31 23:44:34,2020-01-31 23:47:46,3,0.80,1,N,141,141,...,4.5,0.5,0.5,0.00,0.0,0.3,8.30,2.5,0 days 00:03:12,23:30 - 00:00
1956037,6339558,2,2020-01-31 23:01:38,2020-01-31 23:05:49,1,0.74,1,N,48,142,...,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,0 days 00:04:11,23:00 - 23:30


### 3.14 On the `common` dataframe, for each payment type and each interval, determine the average fare amount

Consider the dataset `common` (with the 5 most common destinations for each pickup location). Its rows are **grouped** by the payment type (`payment_type`) and by the pickup time intervals (`pickup_time_interval`), and then the **average fare amount for each interval** is calculated. The `agg()` function compute the mean of the `fare_amount` columns within each time interval.

**Note**: also here is necessary to remove negative values of `fare_amount`.

In [36]:
# Compute the average fare amount for each (payment_type, pickup_time_interval) for common dataset
avg_fare_common = common[common.fare_amount>=0].groupby(['payment_type', 'pickup_time_interval'], 
                                                        observed=False).agg(avg_fare_amount=('fare_amount', 'mean')).reset_index()
avg_fare_common

,payment_type,pickup_time_interval,avg_fare_amount
0,1,00:00 - 00:30,8.587600
1,1,00:30 - 01:00,8.694136
2,1,01:00 - 01:30,8.495154
3,1,01:30 - 02:00,8.025681
4,1,02:00 - 02:30,7.945085
...,...,...,...
235,5,21:30 - 22:00,NaN
236,5,22:00 - 22:30,NaN
237,5,22:30 - 23:00,NaN
238,5,23:00 - 23:30,NaN


### 3.15 Compute the difference of the average fare amount computed in the previous point with those computed at point 9

Calculate the average fare amount for each `payment_type` and `pickup_time_interval` across two datasets: the entire dataset (`dataset`) and the filtered subset (`common`), that contains only the 5 most common destinations for each origin.

First, compute the **average fare amounts** for each **combination** of `payment_type` and `pickup_time_interval` in both datasets. The results are already stored in two separate dataframes: `avg_fare` for the full dataset (computed at `point 9`) and `avg_fare_common` for the filtered subset (computed at `point 14`).

Once having these averages, the next step is to compare them. To do so in a consistent manner, **merge** the two dataframes (`avg_fare` and `avg_fare_common`) based on the common keys: `payment_type` and `pickup_time_interval`. This allows to align the two datasets properly.

After merging, it is easy to calculate the **difference between the average fare amounts from each dataset**. 

In [ ]:
avg_fare_diff = avg_fare_common.rename(   
                                        columns={'avg_fare_amount': 'avg_fare_amount_comm'}                 # Rename columns of avg_fare_common
                                ).merge( 
                                        avg_fare.rename(columns={'avg_fare_amount': 'avg_fare_amount_tot'}),    # Rename columns of avg_fare
                                        on=['payment_type', 'pickup_time_interval'],                            # Apply merge on keys
                                        how='inner')                                                            # Perform an inner join

# Compute the difference between the two columns
avg_fare_diff['average_fare_amount_diff'] = avg_fare_diff['avg_fare_amount_comm'] - avg_fare_diff['avg_fare_amount_tot']
avg_fare_diff

,payment_type,pickup_time_interval,avg_fare_amount_comm,avg_fare_amount_tot,average_fare_amount_diff
0,1,00:00 - 00:30,8.587600,13.868641,-5.281042
1,1,00:30 - 01:00,8.694136,13.473027,-4.778891
2,1,01:00 - 01:30,8.495154,12.822628,-4.327474
3,1,01:30 - 02:00,8.025681,12.358248,-4.332567
4,1,02:00 - 02:30,7.945085,12.008941,-4.063855
...,...,...,...,...,...
235,5,21:30 - 22:00,NaN,NaN,NaN
236,5,22:00 - 22:30,NaN,NaN,NaN
237,5,22:30 - 23:00,NaN,NaN,NaN
238,5,23:00 - 23:30,NaN,NaN,NaN


### 3.16 Compute the ratio between the differences computed in the previous point and those computed in point 9. 

**Note**: you have to compute a ratio for each pair (payment type, interval)

The differences obtained in the previous point (saved within the `avg_fare_diff` dataset in `average_fare_amount_diff` column) are divided by the average fare amount on the overall dataset (`average_fare_amount_tot` column in `avg_fare_diff` dataset) values computed in point 9.

In [38]:
avg_fare_diff['ratio_diff_tot_amounts'] = avg_fare_diff['average_fare_amount_diff'] / avg_fare_diff['avg_fare_amount_tot']
avg_fare_diff

,payment_type,pickup_time_interval,avg_fare_amount_comm,avg_fare_amount_tot,average_fare_amount_diff,ratio_diff_tot_amounts
0,1,00:00 - 00:30,8.587600,13.868641,-5.281042,-0.380790
1,1,00:30 - 01:00,8.694136,13.473027,-4.778891,-0.354701
2,1,01:00 - 01:30,8.495154,12.822628,-4.327474,-0.337487
3,1,01:30 - 02:00,8.025681,12.358248,-4.332567,-0.350581
4,1,02:00 - 02:30,7.945085,12.008941,-4.063855,-0.338402
...,...,...,...,...,...,...
235,5,21:30 - 22:00,NaN,NaN,NaN,NaN
236,5,22:00 - 22:30,NaN,NaN,NaN,NaN
237,5,22:30 - 23:00,NaN,NaN,NaN,NaN
238,5,23:00 - 23:30,NaN,NaN,NaN,NaN


### 3.17 Build chains of trips. Two trips are consecutive in a chain if (a) they have the same VendorID, (b) the pickup location of the second trip is also the dropoff location of the first trip, (c) the pickup time of the second trip is after the dropoff time of the first trip, and (d) the pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip.

**Hint**: Add a column `chain` to the dataset. A chain can have more than two trips.

To prepare the dataset for analysis, a copy of the original dataset is created, retaining only the relevant columns for this task. The selected columns are:

- `VendorID`: The ID representing the vendor.
- `tpep_pickup_datetime`: The timestamp of when the ride was picked up.
- `tpep_dropoff_datetime`: The timestamp of when the ride was dropped off.
- `PULocationID`: The location ID of where the ride was picked up.
- `DOLocationID`: The location ID of where the ride was dropped off.
- `rowID`: A unique identifier for each ride.

After creating the copy, the `PULocationID` and `DOLocationID` columns, originally transformed into categorical, are converted to integer variables. This conversion ensures that location IDs are treated as numerical data, which is necessary for operations like merging and comparisons.

In [39]:
# Keep only necessary columns and create a copy of dataset
dataset_clean = dataset.copy()[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'rowID']]
dataset_clean = dataset_clean.sort_values(by=["VendorID", "tpep_dropoff_datetime", "PULocationID"])

# Convert PULocationID and DOLocationID from category to int
dataset_clean["PULocationID"] = dataset_clean["PULocationID"].astype(int)
dataset_clean["DOLocationID"] = dataset_clean["DOLocationID"].astype(int)

In [40]:
dataset_clean.VendorID.unique()

[1, 2]
Categories (2, Int64): [1, 2]

Two subsets of the dataset are created based on the `VendorID` values, specifically for `VendorID = 1` and `VendorID = 2` (the two categories of the dataset). 

First, a subset for `VendorID = 1` is extracted from the original dataset, and the data is sorted by `VendorID`, `tpep_dropoff_datetime`, and `PULocationID` to ensure that the rides are ordered appropriately.

Similarly, a subset for `VendorID = 2` is created by filtering the dataset for rows where `VendorID = 2`. This subset is also sorted by `VendorID`, `tpep_dropoff_datetime`, and `PULocationID`, following the same order as for `VendorID = 1`.



In [41]:
# Create a subset for VendorID = 1 and another for vendorID = 2
dataset_clean_1 = dataset_clean.copy()[dataset_clean.VendorID == 1] 
dataset_clean_1 = dataset_clean_1.sort_values(by=["VendorID", "tpep_dropoff_datetime", "PULocationID"])

dataset_clean_2 = dataset_clean.copy()[dataset_clean.VendorID == 2] 
dataset_clean_2 = dataset_clean_2.sort_values(by=["VendorID", "tpep_dropoff_datetime", "PULocationID"])

Print headers of both subsets:

In [42]:
dataset_clean_1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,rowID
1487,1,2020-01-01 00:01:40,2020-01-01 00:01:52,79,79,1488
10545,1,2020-01-01 00:00:50,2020-01-01 00:02:32,158,158,10546
5050,1,2020-01-01 00:00:07,2020-01-01 00:03:26,75,75,5051
7236,1,2020-01-01 00:01:55,2020-01-01 00:04:34,141,140,7237
12297,1,2020-01-01 00:01:01,2020-01-01 00:04:46,236,236,12298


In [43]:
dataset_clean_2.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,rowID
782435,2,2003-01-01 00:07:17,2003-01-01 14:16:59,193,193,782436
3136556,2,2008-12-31 23:17:15,2008-12-31 23:32:32,162,263,3136557
3136557,2,2008-12-31 23:35:00,2008-12-31 23:36:53,263,263,3136558
3392619,2,2008-12-31 23:06:13,2008-12-31 23:37:14,144,230,3392620
3136558,2,2008-12-31 23:39:37,2008-12-31 23:55:35,263,143,3136559


A function named `create_chain_df` is defined to generate a **dataset of consecutive trip chains from the input dataset**. This function is designed to work with both subsets created for `VendorID = 1` and `VendorID = 2`.

The function begins by sorting the dataset by `VendorID` and `tpep_pickup_datetime` to ensure that trips are ordered based on pickup times, saving it into a new dataframe called `dataset_sorted`. The sorted dataset is then used for a merge operation, specifically `merge_asof`, which identifies the **valid "next" trip for each ride**. The merge operation **links trips where** the **drop-off datetime** of the first trip and the **pick-up datetime** of the second one **are consecutive**.

Next, the function filters the resulting dataset to retain only the pairs of trips that meet the criteria for being consecutive: the `DOLocationID` of the first trip matches the `PULocationID` of the next trip, and the time between the dropoff of the first trip and the pickup of the next trip is within a 2-minute window.

The `rowID_next`, `PULocationID_next`, and `DOLocationID_next` columns are then converted to integers for consistency. After these steps, the function returns the dataset containing only valid consecutive trip chains (couple of trips).

This function can be applied to both subsets of the data (`dataset_clean_1` and `dataset_clean_2`) to create consecutive trip chains for each vendor.

In [ ]:
# Define a function to create a chain dataset with consecutive pairs of trips
def create_chain_df(dataset_clean):

    # Create a copy sorted by pickup time for merge_asof
    dataset_sorted = dataset_clean.sort_values(by=["VendorID", "tpep_pickup_datetime"])

    # Merge asof to find the valid next trip
    chains = pd.merge_asof(
        dataset_clean,
        dataset_sorted,  # Sorted by pickup time
        by="VendorID",
        left_on="tpep_dropoff_datetime",
        right_on="tpep_pickup_datetime",
        suffixes=("_first", "_next"),
        direction="forward"
    )

    # Filter trips that meet the necessary conditions for being consecutive
    chains = chains[
        (chains["DOLocationID_first"] == chains["PULocationID_next"]) & 
        (chains["tpep_pickup_datetime_next"] - chains["tpep_dropoff_datetime_first"]).dt.total_seconds().between(0, 120)
    ]

    chains['rowID_next'] = chains['rowID_next'].astype(int)
    chains['PULocationID_next'] = chains['PULocationID_next'].astype(int)
    chains['DOLocationID_next'] = chains['DOLocationID_next'].astype(int)

    return chains

In [45]:
# Apply the function on both subsets
chains_1 = create_chain_df(dataset_clean_1)
chains_2 = create_chain_df(dataset_clean_2)

The function `create_trip_chains` is defined to generate chains of consecutive trips from a given dataset. It processes a dataframe (`chain_df`) containing pairs of consecutive trips and **groups them into chains**.

The function begins by initializing an empty dictionary (`chain_dict`) to store the chains and a counter (`chain_counter`) to track the unique chain IDs.

For each row in the dataframe, the function checks whether the `rowID_first` (the ID of the first trip in the pair) is already part of an existing chain. If it is, the `rowID_next` (the ID of the next trip in the chain) is added to that same chain. If no such chain is found, a new chain is created with a new unique chain ID, and the pair of trips is added to this new chain.

Once all rows are processed, the function returns a dictionary where each key is a chain ID, and the associated value is a list of `rowID` values representing the consecutive trips in that chain.

In [46]:
# Define a function to obtain chains of trips
def create_trip_chains(chain_df):
    # Create a dictionary to store chains
    chain_dict = {}
    chain_counter = 0  # Initialize a counter for chain IDs

    # Iterate through the rows of the input dataframe (chain_df)
    for idx, row in chain_df.iterrows():
        row_first = row['rowID_first']  # ID of the first trip in the chain
        row_next = row['rowID_next']   # ID of the next trip in the chain
        
        # Check if rowID_first is already part of an existing chain, and add rowID_next
        found_chain = False  # Flag to check if a chain was found
        for chain_id, row_ids in chain_dict.items():
            if row_first in row_ids:  # If the first trip is already part of a chain
                chain_dict[chain_id].append(row_next)  # Add the next trip to the same chain
                found_chain = True  # Mark that a chain was found
                break
        
        # If no existing chain is found, create a new chain
        if not found_chain:
            chain_counter += 1  # Increment the chain counter
            chain_dict[chain_counter] = [row_first, row_next]  # Create a new chain with the first and next trips

    return chain_dict  # Return the dictionary of chains

Apply the function on chains obtained on the two vendors subsets:

In [47]:
chain_dict_1 = create_trip_chains(chains_1) # Chain of VendorID = 1 trips
chain_dict_2 = create_trip_chains(chains_2) # Chain of VendorID = 2 trips

In [48]:
print('Number of chains of VendorID = 1 trips: ', len(chain_dict_1))

Number of chains of VendorID = 1 trips:  48593


In [49]:
print('Number of chains of VendorID = 2 trips: ', len(chain_dict_2))

Number of chains of VendorID = 2 trips:  98293


Now combine the two dictionaries (`chain_dict_1` and `chain_dict_2`) into a single dictionary (`dict_combined`).

To achieve this, the maximum key value in the first dictionary (`chain_dict_1`) is identified. This value serves as the baseline for updating the keys in the second dictionary (`chain_dict_2`). Each key in `chain_dict_2` is incremented by the maximum key from `chain_dict_1`, ensuring that the keys in both dictionaries do not overlap.

Once the keys of `chain_dict_2` have been updated, the two dictionaries are merged into a single dictionary (`dict_combined`) by using the `**` unpacking operator. This final dictionary contains the **unified chains from both vendors**.

In [50]:
# Unify dictionaries

# Find the max key in the dictionary chain of VendorID = 1
max_key_dict1 = max(chain_dict_1.keys())

# Update keys of dictionary chain of VendorID = 2
chain_dict_2_updated = {max_key_dict1 + key: value for key, value in chain_dict_2.items()}

# Concatenate dictionaries
dict_combined = {**chain_dict_1, **chain_dict_2_updated}

In [51]:
print('Total number of chains: ', len(dict_combined))

Total number of chains:  146886


Analyze chains length:

In [52]:
# Calculate the length of each list (the number of trips per chain)
trip_counts = {key: len(value) for key, value in dict_combined.items()}

# Sort the dictionary based on the number of trips in descending order
sorted_trip_counts = dict(sorted(trip_counts.items(), key=lambda item: item[1], reverse=True))

Display an example of obtained chain (i.e. consider the longest chain present in `sorted_trip_counts`).

**Note**: it can be useful to preserve the same `rowID` order saved within the chain dictionary, to display correctly the sequentiality.

In [53]:
# Extract the first key from the sorted dictionary
first_key = next(iter(sorted_trip_counts))

# Get the list of rowIDs from the first key in dict_combined
row_ids_in_order = dict_combined[first_key]

# Reorder the dataset by matching rowIDs with the order in row_ids_in_order
reordered_dataset = dataset.loc[dataset['rowID'].isin(row_ids_in_order)].set_index('rowID').loc[row_ids_in_order].reset_index()

reordered_dataset[['rowID', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'PULocationID', 'DOLocationID']]

,rowID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2184893,2,2020-01-12 12:52:18,2020-01-12 13:03:34,237,234
1,2197787,2,2020-01-12 13:03:34,2020-01-12 13:06:29,234,234
2,2191331,2,2020-01-12 13:06:29,2020-01-12 13:09:53,234,234
3,2189810,2,2020-01-12 13:09:54,2020-01-12 13:18:49,234,161
4,2196068,2,2020-01-12 13:18:49,2020-01-12 13:25:29,161,163
